In [ ]:
from preamble import *
% matplotlib notebook

# Gaussian Processes
# Check out http://scikit-learn.org/dev/modules/gaussian_process.html

### download data from https://s3.amazonaws.com/tripdata/201307-201402-citibike-tripdata.zip

In [ ]:
from glob import glob
dfs = []
for file in glob("data/citibike/*.csv"):
    dfs.append(pd.read_csv(file))

In [ ]:
data = pd.concat(dfs)

In [ ]:
data.columns

In [ ]:
data.head()


In [ ]:
data['one'] = 1
data['starttime'] = pd.to_datetime(data.starttime)
data = data.set_index("starttime")

In [ ]:
data_resampled = data.groupby("start station id").one.resample("3h").sum()

In [ ]:
per_station = data_resampled.unstack(level=0).fillna(0)

In [ ]:
plt.figure()
per_station[301].plot()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
y = per_station[301].values
X = np.arange(len(y)).reshape(-1, 1)

In [ ]:
gp = GaussianProcessRegressor().fit(X, y)

In [ ]:
plt.figure()
plt.plot(y, label="y")
plt.plot(gp.predict(X), label="preds")
plt.legend()

In [ ]:
gp.kernel_

In [ ]:
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel
gp = GaussianProcessRegressor(alpha=1, normalize_y=True,
                              kernel = 1.0 * RBF(length_scale_bounds=(2, 500)) + 1.0 * RBF(length_scale_bounds=(50, 1000))
                              + 1.0 * RBF(length_scale=100, length_scale_bounds=(2, 500)) * ExpSineSquared(periodicity=8, periodicity_bounds="fixed")  # + 1.0 * WhiteKernel(noise_level=1)
                              + 1.0 * RBF(length_scale=100, length_scale_bounds=(2, 500)) * ExpSineSquared(periodicity=56, periodicity_bounds="fixed")).fit(X[:1500], y[:1500])

In [ ]:
plt.figure()
plt.plot(y, label="y")
plt.plot(gp.predict(X), label="preds")
plt.legend()

In [ ]:
gp.kernel_

# Exercise
Pick a subset of stations from a particular area of the city. Can you use location information to improve the estimates? Can you make predictions for a station given on other stations?